In [13]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [15]:
%run -i ../../zkstats/core.py

In [16]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [17]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = np.array(json.loads(open(data_path, "r").read())['col_name'])
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))
# remember that our create_dummy creates random numbers in range 1 - 30, so need to make sure this example the filter doesnt
# get filter out all data
create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

# # where(element > 20)
gt20_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 20].reshape(1,-1,1)
dummy_theory_output = torch.mean(gt20_dummy_data_tensor)

gt20_data_tensor = data_tensor[data_tensor > 20].reshape(1,-1,1)
theory_output = torch.mean(gt20_data_tensor)


In [33]:
print(gt20_data_tensor.size())
print(dummy_data_tensor.size())
print(data_tensor[0][1][0]<20)

torch.Size([1, 16, 1])
torch.Size([1, 20, 1])
tensor(True)


In [24]:
scales = [5]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [35]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.filtered = nn.Parameter(data = gt20_dummy_data_tensor, requires_grad = False)

    def forward(self,X):
        arb = X.size()[1]>0
        filtered_index = 0
        result = torch.tensor([0]*self.filtered.size()[1])
        filtered_index = filtered_index+1
        # for x_val in X[0]:
        #     result[filtered_index] = x_val[0]==self.filtered[0][filtered_index][0]
        #     filtered_index = torch.where(x_val[0]>20, filtered_index+1, filtered_index)
            
        print('final', filtered_index)
        bool_output = torch.abs(torch.sum(self.filtered)-self.filtered.size()[1]*(self.w))<=torch.abs(0.01*self.filtered.size()[1]*self.w)
        return (torch.logical_and(arb, bool_output), self.w)
        # where part
        filter = (X>20)
        num_fil_X = torch.sum(filter.float())
        fil_X = torch.where(filter, X, 0.0)
        return (torch.abs(torch.sum(fil_X)-num_fil_X*self.w)<=torch.abs(0.01*num_fil_X*self.w), self.w)


verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_8994/1800029062.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([0]*self.filtered.size()[1])


RuntimeError: graph_ == value->owningGraph() INTERNAL ASSERT FAILED at "/Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/ir/ir.cpp":1426, please report a bug to PyTorch. 

In [26]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 20].reshape(1,-1,1)
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.filtered = nn.Parameter(data = gt20_data_tensor, requires_grad = False)

    def forward(self,X):
        # where part
        arb = X.size()[1]>0
        filtered_index = 0
        result = torch.tensor([0]*self.filtered.size()[1])
        for x_val in X[0]:
            result[filtered_index] = x_val[0]==self.filtered[0][filtered_index][0]
            filtered_index = torch.where(x_val[0]>20, filtered_index+1, filtered_index)
        print(filtered_index)
        bool_output = torch.abs(torch.sum(self.filtered)-self.filtered.size()[1]*(self.w))<=torch.abs(0.01*self.filtered.size()[1]*self.w)
        return (torch.logical_and(arb, bool_output), self.w)
        filter = (X>20)
        num_fil_X = torch.sum(filter.float())
        fil_X = torch.where(filter, X, 0.0)

        return (torch.abs(torch.sum(fil_X)-num_fil_X*self.w)<=torch.abs(0.01*num_fil_X*self.w), self.w)


prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory_output:  tensor(50.3750, dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":11,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":1312,"total_assignments":20,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5],"module_sizes":{"kzg":[],"poseidon":[1312,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [27]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 0.25963401794433594 seconds
Theory output:  tensor(50.3750, dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.375
==== Generating Proof ====
proof:  {'instances': [[[16266678685338052443, 7438397994847379675, 5674719498686815871, 2240658077530776119], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [17049824896927915697, 16648112313320311147, 14284844057324796177, 2517225790866338397]]], 'proof': '152f97b2b107c76372d8f3fbed4dba39ae37ef6b1773a88485d93ed36861c30c232f111689976debfdd1ea5a3bca37b96b76b994b1753c33de5ffc8ef3140f68271ec13c48cb354a50d9ee0b8906009a156e8a23f9f0aaff2be1ebd5b4655f940163fc3f2f755d8298c6cfd8fd01a9f13d12423108846780d1237dc9745c63f91d1ed8eb3224fe059f197eb6686829186673a576ff34c45bf9501f1f66cc54bd08b8651f7ae2753362901bb6309f9a72c7445ffcd3b0ed63b00091746e703cbf0c9c106e72e36fce2d39765a494d19b79f0b2342d98f2ade25f3e4aa28e967431ee6cfa836c8550bc6f7cf2bb95576fda960ed1bf83a30144bc35184

In [28]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

50.375